# Assignment 2 Part B

## Harmony Heals 

### Project by RAS (Group 45)
- Rohan Britto (Student ID: 24610990)
- Aaditya Deshmukh (Student ID: )
- Smit Khatri (Student ID: )

## Installing and importing packages

We will start with installing and importing the Musixmatch API that will help us to download song lyrics from Musixmatch server

In [31]:
pip install pymusixmatch

Note: you may need to restart the kernel to use updated packages.


We have set the developer App ID received from the web portal for authentication

In [41]:
from musixmatch import Musixmatch
musixmatch = Musixmatch('7a20605876836eb989b52ac404eaba72') # Musixmatch App ID

Importing pandas and numpy

In [35]:
import pandas as pd
import numpy as np

## Importing the data and analyzing it

Reading the dataset into a pandas dataframe. We manually created a duplicate of the dataset from Kaggle and added a lyrics column to it

In [42]:
df = pd.read_csv('Datasets/muse_v3_lyrics.csv',dtype={'lyrics':object})

In [37]:
df.shape

(90001, 12)

In [38]:
df.head()

,lastfm_url,track,artist,seeds,number_of_emotion_tags,valence_tags,arousal_tags,dominance_tags,mbid,spotify_id,genre,lyrics
0,https://www.last.fm/music/eminem/_/%2527till%2...,'Till I Collapse,Eminem,['aggressive'],6,4.550000,5.273125,5.690625,cab93def-26c5-4fb0-bedd-26ec4c1619e1,4xkOaSrkexMciUUogZKVTS,rap,Not Found
1,https://www.last.fm/music/metallica/_/st.%2banger,St. Anger,Metallica,['aggressive'],8,3.710000,5.833000,5.427250,727a2529-7ee8-4860-aef6-7959884895cb,3fOc9x06lKJBhz435mInlH,metal,St. Anger 'round my neck\nSt. Anger 'round my ...
2,https://www.last.fm/music/rick%2bross/_/speedi...,Speedin',Rick Ross,['aggressive'],1,3.080000,5.870000,5.490000,NaN,3Y96xd4Ce0J47dcalLrEC8,rap,"Legendary, Runners, you know me, o ooooh, o oo..."
3,https://www.last.fm/music/m.i.a./_/bamboo%2bbanga,Bamboo Banga,M.I.A.,"['aggressive', 'fun', 'sexy', 'energetic']",13,6.555071,5.537214,5.691357,99dd2c8c-e7c1-413e-8ea4-4497a00ffa18,6tqFC1DIOphJkCwrjVzPmg,hip-hop,"Road runner, road runner\nGoing hundred miles ..."
4,https://www.last.fm/music/dope/_/die%2bmf%2bdie,Die MF Die,Dope,['aggressive'],7,3.771176,5.348235,5.441765,b9eb3484-5e0e-4690-ab5a-ca91937032a5,5bU4KX47KqtDKKaLM4QCzh,metal,I don't need your forgiveness\nI don't need yo...


As MusixMatch allows downloading only 2000 song lyrics per day and we were targetting a minimum of 12,000 english songs, it took us multiple days to download the lyrics. Each day we had to resume from where we left off on the prior day. To achieve this, we read the rows that had Nan values in the lyrics column and started the download for these.

In [64]:
lyrics_nan = np.where(pd.isnull(df['lyrics']))
lyrics_nan = lyrics_nan[0]
print(lyrics_nan)

[29342 29343 29344 ... 89998 89999 90000]


We set the maximum download limit in a run to 500 as exceeding this led to a timeout error.

Some songs returned a status code 404 (Not Found) or Authorization exception. This was handled in the code and appropriate messages were added so that we could remove them during the data cleaning process.

In [65]:
count = 0
error_count = 0
max_download = 500

for index in lyrics_nan:
    response = musixmatch.matcher_lyrics_get(df.at[index,'track'],df.at[index,'artist'])

    try:
        if response['message']['header']['status_code'] in [404] or response['message']['body']['lyrics']['lyrics_body'] == '':
            lyrics = 'Not Found'
        else:
            lyrics = response['message']['body']['lyrics']['lyrics_body']

    except TypeError:
        print(df.at[index,'track'],": ",df.at[index,'artist'],": ",response)
        lyrics = 'Auth Issue'
        error_count += 1
        if error_count >= 5:
            break

    df.at[index,'lyrics']=lyrics.replace('...\n\n******* This Lyrics is NOT for Commercial use *******','')

    count+=1
    if count >= max_download:
        break

april.#02 :  KASHIWA Daisuke :  {'message': {'header': {'status_code': 401, 'execute_time': 0.0018990039825439}, 'body': []}}


Finally, the dataframe was written back to the same file, so that it could be used again on the next day for downloading the next song lyrics.

In [66]:
df.to_csv('Datasets/muse_v3_lyrics.csv',index=False)